In [13]:
%pip install jupyter-dash dash plotly pandas

In [1]:
import pandas as pd
df = pd.read_csv("DataExtract.csv")
print(df.shape)
print(df.head())

(7099, 38)
   Country  B-G Namespace  Year AQ Model Id        Model Process Id  \
0  Austria  AT.0008.20.AQ  2018   OBE.1.801   OBE.MOP.1.2018.801.96   
1  Austria  AT.0008.20.AQ  2018   OBE.1.802   OBE.MOP.1.2018.802.82   
2  Austria  AT.0008.20.AQ  2018   OBE.1.803  OBE.MOP.1.2018.803.121   
3  Austria  AT.0008.20.AQ  2018   OBE.1.804  OBE.MOP.1.2018.804.110   
4  Austria  AT.0008.20.AQ  2018   OBE.1.805  OBE.MOP.1.2018.805.165   

  Model Area Id       Assessment Type Air Pollutant Air Pollutant Description  \
0     ZON-AT_05  Objective estimation           SO2     Sulphur dioxide (air)   
1     ZON-AT_05  Objective estimation           SO2     Sulphur dioxide (air)   
2     ZON-AT_07  Objective estimation           SO2     Sulphur dioxide (air)   
3     ZON-AT_07  Objective estimation           SO2     Sulphur dioxide (air)   
4     ZON-AT_08  Objective estimation           SO2     Sulphur dioxide (air)   

        Data Aggregation Process  ...  \
0  Annual mean / 1 calendar year  

In [2]:
print(data["Air Pollutant"].unique())
print(data["Country"].unique())
print(data["Year"].unique()[:30])

NameError: name 'data' is not defined

In [3]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import textwrap

# ========== 1. CARGA Y PREPROCESADO ==========
df = pd.read_csv("DataExtract.csv")

cols = [
    "Country", "Year", "Air Pollutant", "Air Pollutant Description",
    "Data Aggregation Process", "Spatial Resolution Description",
    "Temporal Resolution", "Meteorology", "Chemistry", "Emissions", "Topography",
    "Assessment Type", "Administration Level", "Model Application"
]
data = df[cols].copy()
data = data.dropna(subset=["Country", "Year", "Air Pollutant"])
data["Year"] = pd.to_numeric(data["Year"], errors="coerce")
data = data.dropna(subset=["Year"])
data["Year"] = data["Year"].astype(int)

for c in ["Country", "Air Pollutant", "Data Aggregation Process"]:
    data[c] = data[c].astype(str).str.strip()

pollutants = sorted(data["Air Pollutant"].dropna().unique().tolist())
countries = ["Todos los países"] + sorted(data["Country"].dropna().unique().tolist())
ymin, ymax = int(data["Year"].min()), int(data["Year"].max())

# ========== 2. ESTILO GLOBAL ==========
GRAPH_CONFIG = {
    "displaylogo": False,
    "modeBarButtonsToRemove": [
        "zoom2d", "pan2d", "select2d", "lasso2d",
        "autoScale2d", "resetScale2d"
    ],
    "toImageButtonOptions": {
        "format": "png",
        "filename": "eea_air_quality_dashboard",
        "height": 600,
        "width": 1000,
        "scale": 2,
    },
}

def style_fig(fig, title=None):
    """Aplica estilo homogéneo y elegante a todos los gráficos."""
    fig.update_layout(
        title=title or fig.layout.title.text,
        template="plotly_white",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=40, r=20, t=70, b=40),
        font=dict(
            family="Segoe UI, system-ui, -apple-system, sans-serif",
            size=12,
            color="#212529",
        ),
        title_font=dict(
            size=18,
            family="Segoe UI, system-ui, -apple-system, sans-serif",
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.25,
            xanchor="left",
            x=0,
        ),
        hoverlabel=dict(
            bgcolor="white",
            bordercolor="#9ca3af",
            font_size=12,
            font_family="Segoe UI, system-ui, -apple-system, sans-serif",
        ),
    )
    return fig

def format_int(n):
    """Formatea enteros con separador de miles europeo."""
    try:
        return f"{int(n):,}".replace(",", ".")
    except (TypeError, ValueError):
        return "0"

# ========== 3. APP ==========
app = JupyterDash(__name__)
app.title = "EEA Air Quality Models"

CARD_STYLE = {
    "background": "white",
    "borderRadius": "14px",
    "padding": "16px 18px",
    "boxShadow": "0 4px 10px rgba(15, 23, 42, 0.06)",  # sombra más suave
}

KPI_CARD_STYLE = {
    **CARD_STYLE,
    "textAlign": "center",
    "padding": "16px 8px",
}

app.layout = html.Div(
    style={
        "minHeight": "100vh",
        "background": "linear-gradient(135deg, #e0f7fa 0%, #e8f5e9 45%, #bbdefb 100%)",
        "padding": "24px 12px",
    },
    children=[
        html.Div(
            style={"maxWidth": "1300px", "margin": "0 auto"},
            children=[

                # CABECERA
                html.Div(
                    style={"marginBottom": "10px"},
                    children=[
                        html.H1(
                            "EEA Air Quality Models Dashboard",
                            style={
                                "fontFamily": "Segoe UI, system-ui, -apple-system, sans-serif",
                                "fontSize": "32px",
                                "marginBottom": "4px",
                                "color": "#0f172a",
                                "letterSpacing": "0.02em",
                            },
                        ),
                        html.P(
                            "Explora cómo los países europeos modelizan la calidad del aire por contaminante, año y tipo de proceso.",
                            style={
                                "margin": "0",
                                "color": "#1e293b",
                                "fontSize": "14px",
                                "maxWidth": "780px",
                            },
                        ),
                    ],
                ),

                html.Hr(style={"borderColor": "#d1d5db", "margin": "8px 0 18px"}),

                # FILTROS
                html.Div(
                    style={
                        "display": "flex",
                        "gap": "16px",
                        "flexWrap": "wrap",
                        "marginBottom": "18px",
                    },
                    children=[
                        html.Div(
                            style={**CARD_STYLE, "flex": "1 1 260px"},
                            children=[
                                html.Label(
                                    [
                                        "Contaminantes ",
                                        html.Span(
                                            "ⓘ",
                                            title="Selecciona uno o varios contaminantes declarados en los modelos EEA.",
                                            style={
                                                "cursor": "help",
                                                "color": "#6b7280",
                                                "fontSize": "11px",
                                            },
                                        ),
                                    ],
                                    style={
                                        "fontWeight": "600",
                                        "fontSize": "13px",
                                        "color": "#6b7280",
                                    },
                                ),
                                dcc.Dropdown(
                                    id="dd-pol",
                                    options=[
                                        {"label": p, "value": p} for p in pollutants
                                    ],
                                    value=[pollutants[0]],
                                    multi=True,
                                    clearable=False,
                                    style={"marginTop": "6px"},
                                ),
                            ],
                        ),
                        html.Div(
                            style={**CARD_STYLE, "flex": "1 1 260px"},
                            children=[
                                html.Label(
                                    [
                                        "Rango de años ",
                                        html.Span(
                                            "ⓘ",
                                            title="Acota el periodo temporal de los modelos incluidos.",
                                            style={
                                                "cursor": "help",
                                                "color": "#6b7280",
                                                "fontSize": "11px",
                                            },
                                        ),
                                    ],
                                    style={
                                        "fontWeight": "600",
                                        "fontSize": "13px",
                                        "color": "#6b7280",
                                    },
                                ),
                                dcc.RangeSlider(
                                    id="sl-year",
                                    min=ymin,
                                    max=ymax,
                                    step=1,
                                    value=[max(ymin, ymax - 10), ymax],
                                    marks={
                                        y: str(y)
                                        for y in range(
                                            ymin,
                                            ymax + 1,
                                            max(1, (ymax - ymin) // 5 or 1),
                                        )
                                    },
                                    tooltip={
                                        "placement": "bottom",
                                        "always_visible": False,
                                    },
                                    allowCross=False,
                                ),
                            ],
                        ),
                        html.Div(
                            style={**CARD_STYLE, "flex": "1 1 260px"},
                            children=[
                                html.Label(
                                    [
                                        "País ",
                                        html.Span(
                                            "ⓘ",
                                            title="Filtra la vista para un país concreto o analiza todos en conjunto.",
                                            style={
                                                "cursor": "help",
                                                "color": "#6b7280",
                                                "fontSize": "11px",
                                            },
                                        ),
                                    ],
                                    style={
                                        "fontWeight": "600",
                                        "fontSize": "13px",
                                        "color": "#6b7280",
                                    },
                                ),
                                dcc.Dropdown(
                                    id="dd-country",
                                    options=[
                                        {"label": c, "value": c} for c in countries
                                    ],
                                    value="Todos los países",
                                    clearable=False,
                                    style={"marginTop": "6px"},
                                ),
                            ],
                        ),
                    ],
                ),

                # KPIs
                html.Div(
                    style={
                        "display": "flex",
                        "gap": "16px",
                        "flexWrap": "wrap",
                        "marginBottom": "22px",
                    },
                    children=[
                        html.Div(
                            style={**KPI_CARD_STYLE, "flex": "1 1 180px"},
                            children=[
                                html.P(
                                    "Modelos",
                                    style={
                                        "margin": "0 0 4px",
                                        "fontSize": "12px",
                                        "textTransform": "uppercase",
                                        "letterSpacing": "0.12em",
                                        "color": "#9ca3af",
                                        "fontWeight": "600",
                                    },
                                ),
                                html.H2(
                                    id="kpi-models",
                                    style={
                                        "margin": "0",
                                        "fontSize": "30px",
                                        "color": "#111827",
                                    },
                                ),
                                html.P(
                                    "Total de registros de modelos",
                                    style={
                                        "margin": "4px 0 0",
                                        "fontSize": "11px",
                                        "color": "#9ca3af",
                                    },
                                ),
                            ],
                        ),
                        html.Div(
                            style={**KPI_CARD_STYLE, "flex": "1 1 180px"},
                            children=[
                                html.P(
                                    "Países",
                                    style={
                                        "margin": "0 0 4px",
                                        "fontSize": "12px",
                                        "textTransform": "uppercase",
                                        "letterSpacing": "0.12em",
                                        "color": "#9ca3af",
                                        "fontWeight": "600",
                                    },
                                ),
                                html.H2(
                                    id="kpi-countries",
                                    style={
                                        "margin": "0",
                                        "fontSize": "30px",
                                        "color": "#111827",
                                    },
                                ),
                                html.P(
                                    "Países con modelos registrados",
                                    style={
                                        "margin": "4px 0 0",
                                        "fontSize": "11px",
                                        "color": "#9ca3af",
                                    },
                                ),
                            ],
                        ),
                        html.Div(
                            style={**KPI_CARD_STYLE, "flex": "1 1 180px"},
                            children=[
                                html.P(
                                    "Contaminantes",
                                    style={
                                        "margin": "0 0 4px",
                                        "fontSize": "12px",
                                        "textTransform": "uppercase",
                                        "letterSpacing": "0.12em",
                                        "color": "#9ca3af",
                                        "fontWeight": "600",
                                    },
                                ),
                                html.H2(
                                    id="kpi-pollutants",
                                    style={
                                        "margin": "0",
                                        "fontSize": "30px",
                                        "color": "#111827",
                                    },
                                ),
                                html.P(
                                    "Tipos de contaminantes modelizados",
                                    style={
                                        "margin": "4px 0 0",
                                        "fontSize": "11px",
                                        "color": "#9ca3af",
                                    },
                                ),
                            ],
                        ),
                    ],
                ),

                # ========== TABS PRINCIPALES ==========
                dcc.Tabs(
                    id="tabs-main",
                    value="tab-overview",
                    colors={
                        "border": "#e5e7eb",
                        "primary": "#0ea5e9",
                        "background": "transparent",
                    },
                    children=[
                        # --- TAB 1: ANÁLISIS DESCRIPTIVO ---
                        dcc.Tab(
                            label="Análisis descriptivo",
                            value="tab-overview",
                            style={"padding": "10px 16px", "fontWeight": "500"},
                            selected_style={
                                "padding": "10px 16px",
                                "fontWeight": "600",
                                "borderBottom": "3px solid #0ea5e9",
                                "background": "rgba(255,255,255,0.7)",
                            },
                            children=[
                                # FILA 1: MAPA + RANKING
                                html.Div(
                                    style={
                                        "display": "flex",
                                        "gap": "16px",
                                        "flexWrap": "wrap",
                                        "marginTop": "18px",
                                        "marginBottom": "18px",
                                    },
                                    children=[
                                        html.Div(
                                            style={**CARD_STYLE, "flex": "1 1 48%"},
                                            children=[
                                                dcc.Loading(
                                                    type="circle",
                                                    children=[
                                                        dcc.Graph(
                                                            id="fig-map",
                                                            style={"height": "420px"},
                                                            config=GRAPH_CONFIG,
                                                        )
                                                    ],
                                                )
                                            ],
                                        ),
                                        html.Div(
                                            style={
                                                **CARD_STYLE,
                                                "flex": "1 1 48%",
                                                "maxHeight": "440px",
                                                "overflowY": "auto",
                                            },
                                            children=[
                                                dcc.Loading(
                                                    type="circle",
                                                    children=[
                                                        dcc.Graph(
                                                            id="fig-ranking",
                                                            style={"height": "420px"},
                                                            config=GRAPH_CONFIG,
                                                        )
                                                    ],
                                                )
                                            ],
                                        ),
                                    ],
                                ),

                                # FILA 2: EVOLUCIÓN TEMPORAL
                                html.Div(
                                    style={
                                        **CARD_STYLE,
                                        "flex": "1 1 100%",
                                        "marginBottom": "18px",
                                    },
                                    children=[
                                        dcc.Loading(
                                            type="circle",
                                            children=[
                                                dcc.Graph(
                                                    id="fig-trend",
                                                    style={"height": "380px"},
                                                    config=GRAPH_CONFIG,
                                                )
                                            ],
                                        )
                                    ],
                                ),

                                # FILA 3: PROCESOS
                                html.Div(
                                    style={
                                        **CARD_STYLE,
                                        "flex": "1 1 100%",
                                        "marginBottom": "18px",
                                    },
                                    children=[
                                        dcc.Loading(
                                            type="circle",
                                            children=[
                                                dcc.Graph(
                                                    id="fig-process",
                                                    style={"height": "420px"},
                                                    config=GRAPH_CONFIG,
                                                )
                                            ],
                                        )
                                    ],
                                ),
                            ],
                        ),

                        # --- TAB 2: MODELO DE SEGMENTACIÓN ---
                        dcc.Tab(
                            label="Modelo de segmentación",
                            value="tab-model",
                            style={"padding": "10px 16px", "fontWeight": "500"},
                            selected_style={
                                "padding": "10px 16px",
                                "fontWeight": "600",
                                "borderBottom": "3px solid #0ea5e9",
                                "background": "rgba(255,255,255,0.7)",
                            },
                            children=[
                                html.Div(
                                    style={"marginTop": "18px"},
                                    children=[
                                        html.Div(
                                            style={**CARD_STYLE, "marginBottom": "16px"},
                                            children=[
                                                html.H3(
                                                    "Modelo de segmentación por intensidad de modelización",
                                                    style={
                                                        "marginTop": "0",
                                                        "marginBottom": "8px",
                                                        "fontSize": "18px",
                                                        "color": "#111827",
                                                    },
                                                ),
                                                html.P(
                                                    "Para cada país se calcula un score que combina el número de modelos "
                                                    "y el número de contaminantes distintos modelizados. A partir de ese score "
                                                    "se agrupan los países en k segmentos de intensidad.",
                                                    style={
                                                        "fontSize": "13px",
                                                        "color": "#4b5563",
                                                        "marginBottom": "10px",
                                                    },
                                                ),
                                                html.Label(
                                                    "Nº de segmentos (k)",
                                                    style={
                                                        "fontWeight": "600",
                                                        "fontSize": "13px",
                                                        "color": "#6b7280",
                                                    },
                                                ),
                                                dcc.Slider(
                                                    id="sl-k",
                                                    min=2,
                                                    max=6,
                                                    step=1,
                                                    value=3,
                                                    marks={k: str(k) for k in range(2, 7)},
                                                ),
                                            ],
                                        ),
                                        html.Div(
                                            style={**CARD_STYLE, "marginBottom": "16px"},
                                            children=[
                                                dcc.Loading(
                                                    type="circle",
                                                    children=[
                                                        dcc.Graph(
                                                            id="fig-cluster",
                                                            style={"height": "420px"},
                                                            config=GRAPH_CONFIG,
                                                        )
                                                    ],
                                                )
                                            ],
                                        ),
                                        html.Div(
                                            style={**CARD_STYLE},
                                            children=[html.Div(id="txt-k-explanation")],
                                        ),
                                    ],
                                )
                            ],
                        ),

                        # --- TAB 3: COMPOSICIÓN DE MODELOS (TREEMAP) ---
                        dcc.Tab(
                            label="Composición de modelos",
                            value="tab-tree",
                            style={"padding": "10px 16px", "fontWeight": "500"},
                            selected_style={
                                "padding": "10px 16px",
                                "fontWeight": "600",
                                "borderBottom": "3px solid #0ea5e9",
                                "background": "rgba(255,255,255,0.7)",
                            },
                            children=[
                                html.Div(
                                    style={**CARD_STYLE, "marginTop": "18px"},
                                    children=[
                                        html.H3(
                                            "Mapa de árbol de contaminantes y países",
                                            style={
                                                "marginTop": "0",
                                                "marginBottom": "8px",
                                                "fontSize": "18px",
                                                "color": "#111827",
                                            },
                                        ),
                                        html.P(
                                            "El treemap muestra cómo se reparte el número de modelos entre los distintos "
                                            "contaminantes y los países que los reportan. Los rectángulos más grandes "
                                            "representan combinaciones contaminante–país con mayor actividad de modelización.",
                                            style={
                                                "fontSize": "13px",
                                                "color": "#4b5563",
                                                "marginBottom": "10px",
                                            },
                                        ),
                                        dcc.Loading(
                                            type="circle",
                                            children=[
                                                dcc.Graph(
                                                    id="fig-treemap",
                                                    style={"height": "520px"},
                                                    config=GRAPH_CONFIG,
                                                )
                                            ],
                                        ),
                                    ],
                                )
                            ],
                        ),

                        # --- TAB 4: METODOLOGÍA ---
                        dcc.Tab(
                            label="Metodología",
                            value="tab-method",
                            style={"padding": "10px 16px", "fontWeight": "500"},
                            selected_style={
                                "padding": "10px 16px",
                                "fontWeight": "600",
                                "borderBottom": "3px solid #0ea5e9",
                                "background": "rgba(255,255,255,0.7)",
                            },
                            children=[
                                html.Div(
                                    style={**CARD_STYLE, "marginTop": "18px"},
                                    children=[
                                        html.H3(
                                            "Cómo interpretar este dashboard",
                                            style={
                                                "marginTop": "0",
                                                "marginBottom": "8px",
                                                "fontSize": "18px",
                                                "color": "#111827",
                                            },
                                        ),
                                        dcc.Markdown(
                                            """
- **Mapa**: muestra el volumen relativo de modelos reportados por país dentro del rango de años seleccionado.
- **Ranking**: ordena los países según el número de modelos documentados.
- **Evolución temporal**: permite analizar si la actividad de modelización aumenta, se mantiene estable o se reduce.
- **Procesos de agregación**: compara los tipos de procesos de agregación de datos utilizados en los países más activos.
- **Segmentación (k)**: agrupa países en función de un *score* combinado de volumen de modelos y diversidad de contaminantes.
- **Composición de modelos**: el treemap resume visualmente qué contaminantes y qué países concentran más modelos.

Los datos proceden del repositorio de *EEA Air Quality Models* y se han filtrado según los parámetros de la parte superior del dashboard.
                                            """,
                                            style={
                                                "fontSize": "13px",
                                                "color": "#4b5563",
                                            },
                                        ),
                                    ],
                                )
                            ],
                        ),
                    ],
                ),

                # FOOTER
                html.Div(
                    style={
                        "marginTop": "24px",
                        "fontSize": "11px",
                        "color": "#6b7280",
                        "textAlign": "right",
                    },
                    children=[
                        html.Span(
                            "Fuente de datos: EEA Air Quality Models · Dashboard académico."
                        ),
                    ],
                ),
            ],
        )
    ],
)

# ========== 4. CALLBACK ==========
@app.callback(
    [
        Output("kpi-models", "children"),
        Output("kpi-countries", "children"),
        Output("kpi-pollutants", "children"),
        Output("fig-map", "figure"),
        Output("fig-ranking", "figure"),
        Output("fig-trend", "figure"),
        Output("fig-process", "figure"),
        Output("fig-cluster", "figure"),
        Output("fig-treemap", "figure"),
        Output("txt-k-explanation", "children"),
    ],
    [
        Input("dd-pol", "value"),
        Input("sl-year", "value"),
        Input("dd-country", "value"),
        Input("sl-k", "value"),
    ],
)
def update_dashboard(pol_list, year_range, country, k):
    empty_fig = style_fig(go.Figure(), "Sin datos")
    empty_explanation = [
        html.P(
            "Ajusta los filtros y selecciona un valor de k para ver la segmentación de países.",
            style={"fontSize": "13px", "color": "#6b7280", "margin": "0"},
        )
    ]

    if not pol_list or year_range is None or k is None:
        return (
            "0", "0", "0",
            empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig,
            empty_explanation
        )

    y0, y1 = int(year_range[0]), int(year_range[1])
    d = data[
        (data["Air Pollutant"].isin(pol_list))
        & (data["Year"].between(y0, y1))
    ].copy()

    if country != "Todos los países":
        d = d[d["Country"] == country]

    # KPIs
    n_models = len(d)
    n_countries = d["Country"].nunique()
    n_polls = d["Air Pollutant"].nunique()

    # ===== MAPA =====
    geo = d.groupby("Country").size().reset_index(name="Model Count")
    if geo.empty:
        fig_map = style_fig(go.Figure(), "Sin datos para los filtros seleccionados")
    else:
        fig_map = px.choropleth(
            geo,
            locations="Country",
            locationmode="country names",
            color="Model Count",
            color_continuous_scale=px.colors.sequential.YlGnBu,
            scope="europe",
        )
        fig_map = style_fig(
            fig_map,
            "Distribución europea de modelos de calidad del aire (EEA)",
        )
        fig_map.update_layout(
            coloraxis_colorbar=dict(title="Modelos"),
            hovermode="closest",
        )
        fig_map.update_traces(
            hovertemplate="<b>%{location}</b><br>Modelos: %{z}<extra></extra>"
        )

    # ===== RANKING =====
    rank = (
        d.groupby("Country")
        .size()
        .reset_index(name="Model Count")
        .sort_values("Model Count", ascending=False)
        .head(20)
    )
    if rank.empty:
        fig_rank = style_fig(go.Figure(), "Sin datos para ranking")
    else:
        fig_rank = px.bar(
            rank,
            x="Model Count",
            y="Country",
            orientation="h",
        )
        fig_rank.update_layout(yaxis={"categoryorder": "total ascending"})
        fig_rank = style_fig(fig_rank, "Ranking de países por número de modelos")
        fig_rank.update_traces(
            hovertemplate="<b>%{y}</b><br>Modelos: %{x}<extra></extra>"
        )

    # ===== TENDENCIA =====
    trend = (
        d.groupby(["Year", "Air Pollutant"])
        .size()
        .reset_index(name="Model Count")
    )
    if trend.empty:
        fig_trend = style_fig(go.Figure(), "Sin datos para evolución temporal")
    else:
        fig_trend = px.line(
            trend,
            x="Year",
            y="Model Count",
            color="Air Pollutant",
            markers=True,
        )
        fig_trend.update_layout(
            xaxis=dict(dtick=max(1, (y1 - y0) // 10 or 1)),
            hovermode="x unified",
        )
        fig_trend = style_fig(fig_trend, "Evolución del número de modelos por año")
        fig_trend.update_traces(
            hovertemplate="<b>Año %{x}</b><br>Modelos: %{y}<br>Contaminante: %{legendgroup}<extra></extra>"
        )

    # ===== PROCESOS =====
    if "Data Aggregation Process" in d.columns:
        proc = (
            d.groupby(["Country", "Data Aggregation Process"])
            .size()
            .reset_index(name="Count")
        )

        if proc.empty:
            fig_proc = style_fig(go.Figure(), "Sin datos de procesos")
        else:
            top_countries = (
                proc.groupby("Country")["Count"]
                .sum()
                .sort_values(ascending=False)
                .head(10)
                .index
            )
            proc = proc[proc["Country"].isin(top_countries)]

            process_totals = (
                proc.groupby("Data Aggregation Process")["Count"]
                .sum()
                .sort_values(ascending=False)
            )
            top_processes = process_totals.head(5).index

            proc["ProcessGroup"] = proc["Data Aggregation Process"].where(
                proc["Data Aggregation Process"].isin(top_processes),
                "Otros procesos",
            )

            proc["ProcessLabel"] = proc["ProcessGroup"].apply(
                lambda s: "<br>".join(textwrap.wrap(str(s), width=18))
            )

            proc_plot = (
                proc.groupby(["Country", "ProcessLabel"])["Count"]
                .sum()
                .reset_index()
            )

            fig_proc = px.bar(
                proc_plot,
                x="Count",
                y="Country",
                color="ProcessLabel",
                barmode="stack",
            )
            fig_proc = style_fig(
                fig_proc,
                "Procesos de agregación por país (Top 10 países)",
            )
            fig_proc.update_layout(
                xaxis_title="Número de modelos",
                yaxis_title="País",
                yaxis={"categoryorder": "total ascending"},
                legend_title="Proceso de agregación",
                legend=dict(
                    orientation="v",
                    yanchor="top",
                    y=1,
                    xanchor="left",
                    x=1.02,
                    bgcolor="rgba(255,255,255,0.95)",
                    bordercolor="rgba(156,163,175,0.9)",
                    borderwidth=1,
                    font=dict(size=10),
                ),
                margin=dict(l=80, r=230, t=70, b=40),
            )
            fig_proc.update_traces(
                hovertemplate="<b>%{y}</b><br>Proceso: %{legendgroup}<br>Modelos: %{x}<extra></extra>"
            )
    else:
        fig_proc = style_fig(
            go.Figure(),
            "No existe columna 'Data Aggregation Process'",
        )

    # ===== TREEMAP: COMPOSICIÓN DE MODELOS (CONTAMINANTE–PAÍS) =====
    tm = d[["Air Pollutant", "Country"]].dropna()
    if tm.empty:
        fig_treemap = style_fig(
            go.Figure(),
            "Sin datos para el treemap con los filtros seleccionados",
        )
    else:
        tm_group = (
            tm.groupby(["Air Pollutant", "Country"])
            .size()
            .reset_index(name="Count")
        )
        fig_treemap = px.treemap(
            tm_group,
            path=["Air Pollutant", "Country"],
            values="Count",
            color="Air Pollutant",
            color_discrete_sequence=px.colors.qualitative.Set3,
        )
        fig_treemap = style_fig(
            fig_treemap,
            "Composición de modelos por contaminante y país",
        )

    # ===== SEGMENTACIÓN =====
    seg = (
        d.groupby("Country")
        .agg(
            Model_Count=("Country", "size"),
            Pollutant_Count=("Air Pollutant", "nunique"),
        )
        .reset_index()
    )

    if seg.empty or seg["Country"].nunique() < 2:
        fig_cluster = style_fig(
            go.Figure(),
            "No hay suficientes datos para segmentar países",
        )
        explanation_children = empty_explanation
    elif seg["Country"].nunique() < k:
        fig_cluster = style_fig(
            go.Figure(),
            "No hay suficientes países para ese número de segmentos",
        )
        explanation_children = empty_explanation
    else:
        seg["Model_Count_norm"] = seg["Model_Count"] / seg["Model_Count"].max()
        seg["Pollutant_Count_norm"] = seg["Pollutant_Count"] / seg["Pollutant_Count"].max()
        seg["Score"] = 0.7 * seg["Model_Count_norm"] + 0.3 * seg["Pollutant_Count_norm"]

        labels = [f"Segmento {i+1}" for i in range(k)]
        seg["Segment"] = pd.qcut(
            seg["Score"],
            q=k,
            labels=labels,
            duplicates="drop",
        )

        cats = list(seg["Segment"].cat.categories)
        counts = seg["Segment"].value_counts().reindex(cats, fill_value=0)
        total_countries = counts.sum()

        fig_cluster = px.choropleth(
            seg,
            locations="Country",
            locationmode="country names",
            color="Segment",
            scope="europe",
            color_discrete_sequence=px.colors.qualitative.Set3,
        )
        fig_cluster = style_fig(
            fig_cluster,
            f"Segmentación de países por intensidad de modelización (k = {len(cats)})",
        )
        fig_cluster.update_layout(
            legend_title="Segmento",
            hovermode="closest",
        )
        fig_cluster.update_traces(
            hovertemplate="<b>%{location}</b><br>Segmento: %{z}<extra></extra>"
        )

        items = []
        for i, label in enumerate(cats, start=1):
            n_in_segment = int(counts[label])
            pct = round(100 * n_in_segment / total_countries, 1) if total_countries else 0.0

            if i == 1:
                desc = "países con menor score: menor número de modelos y menos variedad de contaminantes (baja intensidad)."
            elif i == len(cats):
                desc = "países con mayor score: mayor número de modelos y mayor variedad de contaminantes (alta intensidad)."
            else:
                desc = "países con score intermedio: nivel medio de modelos y contaminantes (intensidad intermedia)."

            items.append(
                html.Li(
                    f"{label}: {n_in_segment} países (~{pct}%) — {desc}",
                    style={
                        "fontSize": "13px",
                        "color": "#4b5563",
                        "marginBottom": "4px",
                    },
                )
            )

        explanation_children = [
            html.H4(
                "Interpretación de los segmentos",
                style={
                    "marginTop": "0",
                    "marginBottom": "6px",
                    "fontSize": "16px",
                    "color": "#111827",
                },
            ),
            html.P(
                "Cada país recibe un score de intensidad de modelización que combina el volumen de modelos y "
                "la diversidad de contaminantes. Ese score se reparte en k tramos (cuantiles), y cada tramo se "
                "representa como un segmento distinto en el mapa.",
                style={
                    "fontSize": "13px",
                    "color": "#4b5563",
                    "marginBottom": "8px",
                },
            ),
            html.Ul(items),
            html.P(
                "Así, al aumentar k se obtienen grupos más finos (más niveles de intensidad), mientras que con "
                "valores pequeños de k se distinguen solo grandes categorías (por ejemplo, baja / media / alta intensidad).",
                style={
                    "fontSize": "13px",
                    "color": "#4b5563",
                    "marginBottom": "0",
                },
            ),
        ]

    return (
        format_int(n_models),
        format_int(n_countries),
        format_int(n_polls),
        fig_map,
        fig_rank,
        fig_trend,
        fig_proc,
        fig_cluster,
        fig_treemap,
        explanation_children,
    )

# ========== 5. LANZAR SERVIDOR ==========
app.run_server(mode="external", debug=False, port=8050)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2025 19:49:34] "GET /_alive_c05fbaa0-1d65-4e1d-93b0-e0aa1ce0eb78 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [06/Dec/2025 19:49:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:38] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:49:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:50:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:50:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2025 19:50:49] "GET /_dash-dependencies HT